<a href="https://colab.research.google.com/github/SerhiiMis/deep_learning_goit/blob/main/dz_topic_10_SerhiiMishovych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
### 1. Імпорт необхідних бібліотек
!pip install gensim

import numpy as np
import pandas as pd
import re
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [11]:
### 2. Завантаження даних
from google.colab import files
uploaded = files.upload()  # Виберіть файл `spam.csv`

df = pd.read_csv("spam.csv", encoding="latin-1")
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_message'] = df['message'].apply(clean_text)

### Оцінка розподілу цільової змінної
print(df['label'].value_counts(normalize=True) * 100)

### Розбиття на навчальний та валідаційний набори
y = df['label_num']
X_train, X_test, y_train, y_test = train_test_split(df['clean_message'], y, test_size=0.2, random_state=42, stratify=y)

Saving spam.csv to spam (1).csv
label
ham     86.593683
spam    13.406317
Name: proportion, dtype: float64


In [12]:
### 3. Векторизація BoW та TF-IDF
bow_vectorizer = CountVectorizer(max_features=5000, ngram_range=(1,2))
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [13]:
### 4. Використання попередньо навчених ембедингів (Word2Vec)
w2v_model = api.load("word2vec-google-news-300")

def text_to_vector(text, model, vector_size=300):
    words = text.split()
    word_vectors = [model[word] for word in words if word in model]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(vector_size)

X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_test_w2v = np.array([text_to_vector(text, w2v_model) for text in X_test])

In [14]:
### 5. Навчання моделей
model_bow = LogisticRegression(max_iter=1000, random_state=42)
model_bow.fit(X_train_bow, y_train)

model_tfidf = LogisticRegression(max_iter=1000, random_state=42)
model_tfidf.fit(X_train_tfidf, y_train)

model_w2v = LogisticRegression(max_iter=1000, random_state=42)
model_w2v.fit(X_train_w2v, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [15]:
### 6. Оцінка моделей
y_pred_bow = model_bow.predict(X_test_bow)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
y_pred_w2v = model_w2v.predict(X_test_w2v)

accuracy_bow = accuracy_score(y_test, y_pred_bow)
auc_bow = roc_auc_score(y_test, y_pred_bow)

accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
auc_tfidf = roc_auc_score(y_test, y_pred_tfidf)

accuracy_w2v = accuracy_score(y_test, y_pred_w2v)
auc_w2v = roc_auc_score(y_test, y_pred_w2v)

print("BoW - Accuracy:", accuracy_bow, "AUC:", auc_bow)
print("TF-IDF - Accuracy:", accuracy_tfidf, "AUC:", auc_tfidf)
print("Word2Vec - Accuracy:", accuracy_w2v, "AUC:", auc_w2v)

BoW - Accuracy: 0.9802690582959641 AUC: 0.9290126029985967
TF-IDF - Accuracy: 0.9730941704035875 AUC: 0.9021669654147039
Word2Vec - Accuracy: 0.9668161434977578 AUC: 0.9155724151347145


In [ ]:
### 7. Аналіз та висновки
# Порівняння результатів
# BoW показав найкращі результати з точністю 98.03% та AUC 0.929, що вказує на ефективність цього підходу.
# TF-IDF також показав гарний результат (97.31% точності), проте має трохи нижчий AUC (0.902), що свідчить про слабшу здатність розрізняти класи.
# Word2Vec дав трохи нижчий результат (96.68% точності), але AUC 0.916 вказує, що модель все ще добре справляється із класифікацією.
# Загалом, BoW є найкращим підходом для цього завдання.
# Подальші покращення можуть включати використання більш складних моделей (наприклад, LSTM або BERT) або додаткову обробку даних.